In [4]:
import tensorflow as tf
from tensorflow import keras

在网络训练的过程中，通过Web端远程监控网络的训练进度，可视化网络的训练结果，对于提高开发效率和实现远程监控是非常重要的。TensorFlow提供了一个专门的可视化工具，叫做TensorBoard，它通过TensorFlow将监控数据写入到文件系统，并利用Web后端监控对应的文件目录，从而可以允许用户从远程查看网络的监控数据

要使用tensorboard工具监控网络训练进度，要从模型端和浏览器端入手

## 1.模型端

模型端需要创建写入监控数据的Summary类，并指定监控数据的写入目录：

In [5]:
log_dir='./log'
summary_writer=tf.summary.create_file_writer(log_dir)

例如在前向计算完后，对于误差这种数据，需要监控，并赋予一个ID

下面结合之前构建的网络做例子来演示：

In [6]:
def preprocess(x,y):
    # 调用此函数会自动传入x，y
    # 标准化到0~1
    x=tf.cast(x,dtype=tf.float32)/255.
    x=tf.reshape(x,[-1,28*28]) # 打平
    y=tf.cast(y,dtype=tf.int32) # 转换成整型张量
    y=tf.one_hot(y,depth=10) # 进行one-hot编码
    return x,y

def load_data():
    # 加载MNIST
    (x,y),(x_val,y_val)=keras.datasets.mnist.load_data()
    batchsz=512
    # 构建数据集对象
    train_dataset=tf.data.Dataset.from_tensor_slices((x,y))
    train_dataset=train_dataset.shuffle(1000)
    #批量训练
    train_dataset=train_dataset.batch(batchsz)
    train_dataset=train_dataset.map(preprocess)
    train_dataset=train_dataset.repeat(20)

    # 加载验证/测试集
    val_dataset=tf.data.Dataset.from_tensor_slices((x_val,y_val))
    val_dataset=val_dataset.shuffle(1000).batch(batchsz).map(preprocess)
    return train_dataset,val_dataset

class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1=keras.layers.Dense(256,activation='relu')
        self.fc2=keras.layers.Dense(128,activation='relu')
        self.fc3=keras.layers.Dense(10,activation='relu')

    def call(self, inputs, training=None, mask=None):
        x=self.fc1(inputs)
        x=self.fc2(x)
        x=self.fc3(x)
        return x

model=MyModel()
model.build(input_shape=(None,28*28))

optimizer=keras.optimizers.RMSprop(0.001)
train_dataset,val_dataset=load_data()
loss_meter=keras.metrics.Mean() # 1.新建平均测量器

for epoch in range(10):
    for step,(x,y) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            out=model(x)
            loss=tf.losses.MSE(y,out)
            mean_mse_loss=tf.reduce_mean(loss)
            loss_meter.update_state(loss) # 2.写入数据，记录采样的数据
            '''
            summary记录
            '''
            with summary_writer.as_default(): # 写入环境
                # 当前时间戳step上的loss写入到id为train-loss数据库中
                tf.summary.scalar('train-loss',float(mean_mse_loss),step=step)
        grads=tape.gradient(mean_mse_loss,model.trainable_variables)
        optimizer.apply_gradients(zip(grads,model.trainable_variables))

        if step%500==0:
            print(f'epoch[{epoch}]-step[{step}] loss:{loss_meter.result()}') # 3.读取统计信息
            loss_meter.reset_states() # 4.清零测量器

epoch[0]-step[0] loss:0.1403273046016693
epoch[0]-step[500] loss:0.020966259762644768
epoch[0]-step[1000] loss:0.003918889909982681
epoch[0]-step[1500] loss:0.0020578422117978334
epoch[0]-step[2000] loss:0.0012696660123765469
epoch[1]-step[0] loss:0.0008957092650234699
epoch[1]-step[500] loss:0.0006887515773996711
epoch[1]-step[1000] loss:0.0005277349846437573
epoch[1]-step[1500] loss:0.0004293204110581428
epoch[1]-step[2000] loss:0.0003579477488528937
epoch[2]-step[0] loss:0.0003179462219122797
epoch[2]-step[500] loss:0.0002832511381711811
epoch[2]-step[1000] loss:0.00025106361135840416
epoch[2]-step[1500] loss:0.0002210118982475251
epoch[2]-step[2000] loss:0.00019726170285139233
epoch[3]-step[0] loss:0.00019218225497752428
epoch[3]-step[500] loss:0.00017839492647908628
epoch[3]-step[1000] loss:0.00016368877550121397
epoch[3]-step[1500] loss:0.00014910355093888938
epoch[3]-step[2000] loss:0.0001413038553437218
epoch[4]-step[0] loss:0.00013447608216665685
epoch[4]-step[500] loss:0.0001

## 2.浏览器端
为了查看可视化的这些数据，需要在终端运行`tensorboard --logdir path`

```bash
tensorboard --logdir log --host=0.0.0.0
```

查看结果如下：

![](https://github.com/zfhxi/Learn_tensorflow/blob/master/ch08-Keras%E9%AB%98%E5%B1%82%E6%8E%A5%E5%8F%A3/img/01.png?raw=true)

In [ ]:
import os
pid=os.getpid()
!kill -9 $pid